In [11]:
import numpy as np
import pandas as pd
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
import pymannkendall as mk
import random

%matplotlib inline

In [12]:
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(10,6))

In [17]:
def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    print(sf.shapes())
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

In [18]:
def read_shapefile_by_year(year):
    shp_path = "Tracts_WUI_ByYear/tracts_wui_" + str(year) + ".shp"
    sf = shp.Reader(shp_path)
    return read_shapefile(sf)

In [19]:
df = read_shapefile_by_year(2019)
print(df.shape)
df.head()

Shapes: [<shapefile.Shape object at 0x00000142B8DCCFA0>, <shapefile.Shape object at 0x00000142B8DE7070>, <shapefile.Shape object at 0x00000142B8DE7100>, <shapefile.Shape object at 0x00000142B8DE7190>, <shapefile.Shape object at 0x00000142B8DE7220>, <shapefile.Shape object at 0x00000142B8DE72B0>, <shapefile.Shape object at 0x00000142B8DE7310>, <shapefile.Shape object at 0x00000142B8DE7370>, <shapefile.Shape object at 0x00000142B8DE7400>, <shapefile.Shape object at 0x00000142B8DE7460>, <shapefile.Shape object at 0x00000142B8DE74F0>, <shapefile.Shape object at 0x00000142B8DE7550>, <shapefile.Shape object at 0x0000014294645D30>, <shapefile.Shape object at 0x00000142944B7D00>, <shapefile.Shape object at 0x00000142846443D0>, <shapefile.Shape object at 0x0000014284644550>, <shapefile.Shape object at 0x0000014284661730>, <shapefile.Shape object at 0x0000014284661640>, <shapefile.Shape object at 0x00000142A447F520>, <shapefile.Shape object at 0x00000142A447F2B0>, <shapefile.Shape object at 0x00

(3523, 19)


,STATEFP10,COUNTYFP10,TRACTCE10,id,NAME10,NAMELSAD10,MTFCC10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geo_id,census_tra,county,estimate,error,year,coords
0,06,037,500403,6037500403,5004.03,Census Tract,G5020,S,2057389,189585,+34.0073290,-118.0660305,1400000US06037500403,5004.03,Los Angeles County,3829,271,2019,"[(-118.072722, 33.995982999999995), (-118.0721..."
1,06,065,045103,6065045103,451.03,Census Tract,G5020,S,23058405,383371,+33.7382362,-116.4469086,1400000US06065045103,451.03,Riverside County,3557,347,2019,"[(-116.477924, 33.757493), (-116.477901, 33.75..."
2,06,065,046404,6065046404,464.04,Census Tract,G5020,S,10645119,0,+33.6510929,-117.2771726,1400000US06065046404,464.04,Riverside County,6661,471,2019,"[(-117.3002, 33.660089), (-117.29991799999999,..."
3,06,065,045108,6065045108,451.08,Census Tract,G5020,S,2683462,0,+33.7251292,-116.3737209,1400000US06065045108,451.08,Riverside County,6767,898,2019,"[(-116.391385, 33.724388999999995), (-116.3913..."
4,06,037,122122,6037122122,1221.22,Census Tract,G5020,S,1017265,15394,+34.2197648,-118.3736105,1400000US06037122122,1221.22,Los Angeles County,2728,282,2019,"[(-118.379477, 34.223303), (-118.377439, 34.22..."


In [16]:
df.coords

0       [(-118.072722, 33.995982999999995), (-118.0721...
1       [(-116.477924, 33.757493), (-116.477901, 33.75...
2       [(-117.3002, 33.660089), (-117.29991799999999,...
3       [(-116.391385, 33.724388999999995), (-116.3913...
4       [(-118.379477, 34.223303), (-118.377439, 34.22...
                              ...                        
3518    [(-118.378042, 34.645081999999995), (-118.3778...
3519    [(-117.97774799999999, 34.558136999999995), (-...
3520    [(-118.542543, 34.486174), (-118.542472, 34.48...
3521    [(-118.454312, 34.592611999999995), (-118.4541...
3522    [(-118.50267099999999, 34.418503), (-118.50228...
Name: coords, Length: 3523, dtype: object

In [6]:
def getDataByName(name):
    years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
    estimate_list = []
    for year in years:
        data_for_year = read_shapefile_by_year(year)
        estimate_list.append(data_for_year[data_for_year.NAME10 == name].estimate)
    return estimate_list

In [7]:
d = getDataByName("5004.03")

In [8]:
d

[0    4067
 Name: estimate, dtype: int64,
 0    4360
 Name: estimate, dtype: int64,
 0    4464
 Name: estimate, dtype: int64,
 0    4404
 Name: estimate, dtype: int64,
 0    4536
 Name: estimate, dtype: int64,
 0    4483
 Name: estimate, dtype: int64,
 0    4193
 Name: estimate, dtype: int64,
 0    4032
 Name: estimate, dtype: int64,
 0    3896
 Name: estimate, dtype: int64,
 0    3829
 Name: estimate, dtype: int64]

In [14]:
def getAllData():
    years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
    all_data = {}
    for year in years:
        data_for_year = read_shapefile_by_year(year)
        for index, row in data_for_year.iterrows():
            key = row["id"]
            value = row["estimate"]
            
            if not key in all_data:
                all_data[key] = []
            
            all_data[key].append(value)
    return all_data

In [15]:
all_data = getAllData()
all_data

{'6037500403': [4067, 4360, 4464, 4404, 4536, 4483, 4193, 4032, 3896, 3829],
 '6065045103': [4107, 4121, 4218, 4031, 3909, 3553, 3403, 3488, 3403, 3557],
 '6065046404': [4723, 5174, 5293, 5331, 5967, 6427, 6782, 7279, 6328, 6661],
 '6065045108': [5082, 5481, 5458, 5512, 6069, 6381, 6815, 6783, 6785, 6767],
 '6037122122': [1964, 2251, 2071, 2118, 2238, 2419, 2383, 2610, 2652, 2728],
 '6065045109': [4132, 3865, 3927, 3945, 3631, 3416, 3576, 3564, 3637, 4137],
 '6065045110': [5064, 5077, 5170, 4987, 5205, 5411, 5294, 5295, 5938, 6290],
 '6065042514': [3078, 3029, 2820, 2965, 2811, 3019, 3167, 3216, 3094, 3467],
 '6065043807': [5199, 5231, 5552, 5770, 5893, 5907, 6409, 6526, 6866, 7040],
 '6065043900': [6438, 5955, 6002, 6097, 5802, 5990, 6442, 6580, 6704, 6698],
 '6037137301': [2237, 2117, 2122, 2284, 2236, 2276, 2356, 2344, 2354, 2271],
 '6065045215': [8491, 8932, 9083, 9245, 9360, 9739, 9735, 10346, 9846, 9761],
 '6065045216': [1320, 1673, 1795, 1938, 1902, 1805, 1913, 2233, 2280, 2407]

In [16]:
sum_ = 0
count = 0
for k in all_data:
    sum_ += len(all_data[k])
    count += 1
print("Avg Length of List:", sum_ / count)
print("Num Items:", len(all_data))

Avg Length of List: 10.0
Num Items: 3523


In [17]:
results = {}
avg_p = []
for k in all_data:
    results[k] = mk.original_test(all_data[k])
    avg_p.append(results[k].p)
np.mean(avg_p)

0.2255809363996635

In [18]:
results

{'6037500403': Mann_Kendall_Test(trend='no trend', h=False, p=0.15240628395674927, z=-1.4310835055998654, Tau=-0.37777777777777777, s=-17.0, var_s=125.0, slope=-66.375, intercept=4575.1875),
 '6065045103': Mann_Kendall_Test(trend='decreasing', h=True, p=0.015322241055100827, z=-2.4246715773614613, Tau=-0.6222222222222222, s=-28.0, var_s=124.0, slope=-94.42857142857143, intercept=4157.928571428572),
 '6065046404': Mann_Kendall_Test(trend='increasing', h=True, p=0.002357527595795439, z=3.041052449399714, Tau=0.7777777777777778, s=35.0, var_s=125.0, slope=311.0, intercept=4748.0),
 '6065045108': Mann_Kendall_Test(trend='increasing', h=True, p=0.004207551285491773, z=2.862167011199731, Tau=0.7333333333333333, s=33.0, var_s=125.0, slope=212.875, intercept=5267.0625),
 '6037122122': Mann_Kendall_Test(trend='increasing', h=True, p=0.0012821837418053317, z=3.219937887599697, Tau=0.8222222222222222, s=37.0, var_s=125.0, slope=84.88888888888889, intercept=1935.0),
 '6065045109': Mann_Kendall_Tes

In [27]:
data2010 = read_shapefile_by_year(2010)

# id
# trend, p, z, slope, intercept
trendData = pd.DataFrame(columns = ["id", "county", "trend", "p", "z", "slope", "intercept", "coords"])

for index, row in data2010.iterrows():
    newRow = {
        "id": row["id"],
        "coords": row["coords"],
        "county": row["county"]
    }
    test = results[newRow["id"]]
    newRow["trend"] = test.trend
    newRow["p"] = test.p
    newRow["z"] = test.z
    newRow["slope"] = test.slope
    newRow["intercept"] = test.intercept
    trendData = trendData.append(newRow, ignore_index=True)

trendData

,id,county,trend,p,z,slope,intercept,coords
0,6037500403,Los Angeles County,no trend,0.152406,-1.431084,-66.375000,4575.187500,"[(-118.072722, 33.995982999999995), (-118.0721..."
1,6065045103,Riverside County,decreasing,0.015322,-2.424672,-94.428571,4157.928571,"[(-116.477924, 33.757493), (-116.477901, 33.75..."
2,6065046404,Riverside County,increasing,0.002358,3.041052,311.000000,4748.000000,"[(-117.3002, 33.660089), (-117.29991799999999,..."
3,6065045108,Riverside County,increasing,0.004208,2.862167,212.875000,5267.062500,"[(-116.391385, 33.724388999999995), (-116.3913..."
4,6037122122,Los Angeles County,increasing,0.001282,3.219938,84.888889,1935.000000,"[(-118.379477, 34.223303), (-118.377439, 34.22..."
...,...,...,...,...,...,...,...,...
3518,6037901213,Los Angeles County,no trend,0.474274,0.715542,15.500000,3790.750000,"[(-118.378042, 34.645081999999995), (-118.3778..."
3519,6037910001,Los Angeles County,increasing,0.012266,2.504396,74.428571,5775.071429,"[(-117.97774799999999, 34.558136999999995), (-..."
3520,6037920011,Los Angeles County,no trend,1.000000,0.000000,0.000000,239.500000,"[(-118.542543, 34.486174), (-118.542472, 34.48..."
3521,6037920012,Los Angeles County,no trend,0.126849,-1.526645,-13.500000,1100.750000,"[(-118.454312, 34.592611999999995), (-118.4541..."


In [28]:
outpath = "Tracts_WUI_ByYear/trends.shp"

geoTrendData = gpd.GeoDataFrame(trendData, geometry="coords")

AttributeError: 'DataFrame' object has no attribute 'to_file'

In [20]:
# Liz did some stuff here that I'm not able to run on my computer, so this file is outdated